In [3]:
! pip install torchtext
! python3 -m spacy download en_core_web_sm
! pip install transformers

Defaulting to user installation because normal site-packages is not writeable
/bin/python3: No module named spacy
Defaulting to user installation because normal site-packages is not writeable


In [7]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [8]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:12<00:00, 6.55MB/s]


In [10]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['This', 'is', 'a', 'really', 'interesting', 'movie', '.', 'It', 'is', 'an', 'action', 'movie', 'with', 'comedy', 'mixed', 'in', '.', 'Foxx', 'teams', 'up', 'with', 'comedian', 'Epps', 'in', 'this', 'movie', 'to', 'give', 'it', 'a', 'comedic', 'spin', '.', 'It', 'will', 'keep', 'you', 'wondering', 'what', 's', 'going', 'to', 'happen', 'to', 'Foxx', 'next', '.', 'It', 'was', 'a', 'well', 'shot', 'movie', ',', 'the', 'director', 'used', 'the', 'right', 'colors', 'in', 'this', 'movie(dark', 'blue', 'colors', ')', 'to', 'give', 'it', 'the', 'right', 'kind', 'of', 'feel', '.', 'Kimberly', 'Elise', 'also', 'starred', 'in', 'this', 'movie', 'and', 'it', 'is', 'always', 'a', 'pleasure', 'to', 'see', 'her', 'on', 'the', 'big', 'screen', '.', 'She', 'plays', 'her', 'role', 'well', '.', 'Even', 'Jamie', 'Kennedy', 'is', 'in', 'this', 'movie', '.', 'It', "'s", 'worth', 'seeing', 'it', 'you', 'have', "n't", 'seen', 'it', 